In [21]:
import os

import pandas as pd
from dotenv import load_dotenv
from utils.course import ChatOpenAI
from tqdm import tqdm

In [22]:
tqdm.pandas()

In [7]:
load_dotenv()
api_key_course = os.getenv("API_COURSE")

In [8]:
# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=api_key_course)

__Задача:__
- Для каждого сообщения необходимо определить, является ли оно спамом (1) или не спамом (0) и записать эту информацию в столбец is_spam.
Что на выходе? csv файл, содержащий два столбца - text_id и is_spam.Что использовать? В этой задаче не подразумевается использование сторонних ML библиотек, кроме API ChatGPT и библиотеки pandas для обработки данных.
- Замечание: Точность вашего решения (доля верных ответов) должна быть не ниже 75%.

In [11]:
df = pd.read_csv("../data/2_2_8_100_crypto_messages.csv")

In [12]:
df

,text_id,text
0,143,Удаленная работа. От нас: Зп от 2500$
1,179,Что вы думаете о концепции децентрализованных ...
2,147,Удалённый заработок. Время 1-2часа в день. Мож...
3,157,Меня всегда интересовал вопрос безопасности хр...
4,168,"Я слышал, что некоторые банки начали экспериме..."
...,...,...
95,209,Работа в сфере криптовалютной журналистики или...
96,200,Какие криптовалюты вы видите в будущем как осн...
97,229,Исследование и анализ рынка криптовалют также ...
98,101,Набираю людей в команду по межбиржевой торговл...


In [13]:
from langchain import PromptTemplate

In [14]:
## Точность вашего решения: 0.72. Нужно как минимум 0.75
# template = """Внимательно прочитай вопрос и опираясь на контекст ниже определи тип вопроса.
# Если сообщение несет рекламный характер - это спам, пометь его цифрой 1.
# Если сообщение полезное и не рекламирует никакие товары или услуги, то это не спам. Пометь его цифрой 0.
#
# Context: Удалённый заработок. Время 1-2часа в день. Можно без опыта всему обучим. заработок в долларах. Возраст 20+
#
# Question: {query}
#
# Answer: """
#
# prompt_template = PromptTemplate(
#     input_variables=["query"],
#     template=template
# )

In [36]:
# Отлично! Вы побили порог в 0.75: ваша точность 0.95.
template = """
Внимательно прочитай вопрос и определи является ли данное сообщение спамом, рекламой или мусором.
Если да, то поставсь такому сообщению цифру 1. Если же сообщение несет полезную информацию (вопрос или предложение), то поставь такому сообщению цифру 0.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [37]:
# df["is_spam"] = df["text"].progress_apply(lambda x: llm.predict(prompt_template.format(query=x)))

100%|██████████| 100/100 [00:45<00:00,  2.21it/s]


In [39]:
# df[["text_id", "is_spam"]].to_csv("../data/2_2_8_100_solution.csv", index=False)

## 2_2_9

In [46]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

In [40]:
df = pd.read_csv("../data/2_2_9_vacancies_messages_50.csv")

In [41]:
df

,text_id,text
0,9,#вакансия #vacancy #Python #удаленка #flask #r...
1,31,#ВАКАНСИЯ #Системный_аналитик #РФ\n \nАКЦИОНЕ...
2,28,#вакансия #vacancy #job #senior #data #DB #dat...
3,49,#вакансия #fulltime #remote \n\n🔎 Ищем Руковод...
4,18,#vacancy #job #analyst #travel #sirenatravel #...
5,48,#вакансия #fulltime #remote #lookfor #devops\n...
6,41,#вакансия #senior #middle+ #lead #удаленка #оф...
7,6,#job #python #django #javascript #react #fulls...
8,17,#вакансия #vacancy #middle #senior #remote #уд...
9,4,Вакансия: Разработчик (Vue.js / Golang) для на...


In [61]:
template = """Из следующего текста извлеки информацию:

job_title: ...

company: ...

salary: ...

tg: ...

grade: ...

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)

In [62]:
# # Результат: 128 из 250
# job_title_schema = ResponseSchema(
#     name="job_title",
#     description="Как указано в описании вакансии, на том же языке. Проверяем полное совпадение, без учета регистра. Если написан грейд, его нужно убрать (например, Senior Python developer -> Python developer, C++ разработчик (middle, senior) ->C++ разработчик)."
# )
#
#
# company_schema = ResponseSchema(
#     name="company",
#     description="Как указано в описании вакансии, на том же языке. Проверяем полное совпадение, без учета регистра. Указываем только название (не пишем финтех, крупная компания, мобильная игра и т.д.)."
# )
#
#
# salary_schema = ResponseSchema(
#     name="salary",
#     description="Числа пишем без пробелов, не пишем тыс. или к.(умножаем в таком случае на 1000), не пишем фикс, плюшки, премии, % от продаж, техника и т.д. Не пишем net, gross, на руки и т.д. После числа (диапазона чисел) указываем валюту руб. или $ после пробела. Если указан диапазон, то пишем его через тире, около тире пробелы не ставим (например, 100к-150к рублей -> 100000-150000 руб.). Если указана только нижняя граница (от 2000$ или 100000+руб), то пишем это значение, используя слово от (например, 100к+руб -> от 100000 руб.). Если указана только верхняя граница, то пишем это значение, используя слово до (например, до 100к руб -> до 100000 руб.). Если зарплата указана за час, то в конце добавляем в час."
# )
#
#
# tg_schema = ResponseSchema(
#     name="tg",
#     description="Указываем контакт в телеграмм, используя @. Проверяем полное совпадение, с учетом регистра. Если указано несколько контаков, то указываем их через запятую (не забывая про пробел после запятой)."
# )
#
#
# grade_schema = ResponseSchema(
#     name="grade",
#     description="Возможные значения intern, junior, junior+, middle, middle+, senior, lead. Если указано несколько значений, то пишем их через запятую в порядке возрастания."
# )
#
# response_schemas = [
#     job_title_schema,
#     company_schema,
#     salary_schema,
#     tg_schema,
#     grade_schema
# ]

In [76]:
job_title_schema = ResponseSchema(
    name="job_title",
    description="""На какую позицию/вакансию ищут кандидата?
Не указывай уровень, просто напиши название позиции,
например, для названия 'Senior Python developer' - это просто 'Python developer', а 'C++ разработчик (middle, senior)' 'C++ разработчик'
Из названия удали grade, то есть уровень. Название позиции не должно содержать слов junior, middle, senior, lead.
Если эта информация напрямую не указана в вакансии, ответь 'не знаю'.
""",
)

company_schema = ResponseSchema(
    name="company",
    description="""Напиши название компании, для которой ищут кандидата.
Название напиши на том же языке, что и в тексте, точно так же, как указано в вакансии.
В качестве ответа требуется только название, без дополнительных пояснений (например, что это крупная компания, финтех или прочее)
Если эта информация напрямую не указана в вакансии, ответь 'не знаю'.""",
)

salary_schema = ResponseSchema(
    name="salary",
    description="""Укажи предполагаемую в вакансии зарплату в соответствии со следующими правилами:
1) Не пиши 'тыс.' или 'к.', если такие конструкции встречаются в тексте, умножай на 1000 указанное до них число.
2) Не пиши 'net', 'gross', 'на руки' и подобные им.
3) После числа (или диапазона чисел, если указан диапазон возможных зарплат), укажи валюту, в которой будут платить. Если это рубли, то напиши после числа (чисел) 'руб.', а если доллары - '$'  ДО ДОЛЛАРОВ НЕ ЗАБУДЬ ПОСТАВИТЬ ПРОБЕЛ.
4) Если указан диапазон, то напиши его через тире, около тире пробелы не ставь (например, '100к-150к рублей' -> '100000-150000 руб.').
5) Если указана только нижняя граница (от 2000$ или 100000+руб), то напиши это значение, используя слово 'от' (например, '100к+руб' -> 'от 100000 руб.').
6) Если указана только верхняя граница, то напиши это значение, используя слово 'до' (например, 'до 100к руб' -> 'до 100000 руб.').
7) Если зарплата указана за час, то в конце добавь 'в час'.
Если эта информация напрямую не указана в вакансии, ответь 'не знаю'.
""",
)

tg_schema = ResponseSchema(
    name="tg",
    description="""Укажи контакт для связи. Контакт перепиши в точности как в тексте, с учетом регистра, при необходимости добавив '@' перед ним (такой символ для каждого контакта должен быть один)
Если указано несколько контаков, то укажи их через запятую (не забывая про пробел после запятой).
Если эта информация напрямую не указана в вакансии, ответь 'не знаю'.""",
)

grade_schema = ResponseSchema(
    name="grade",
    description="""Напиши уровень, на который ищут кандидата. Возможные значения: 'intern', 'junior', 'junior+', 'middle', 'middle+', 'senior', 'lead'.
Если указано несколько значений, то напиши их через запятую в порядке возрастания.
Если эта информация напрямую не указана в вакансии, ответь 'не знаю'.
""",
)

response_schemas = [
    job_title_schema,
    company_schema,
    salary_schema,
    tg_schema,
    grade_schema,
]

In [77]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas) # Создаём парсер и подаём в него список со схемами
format_instructions = output_parser.get_format_instructions() # Получаем инструкции по форматированию ответа

In [78]:
dict_list = [] # Список из словарей, где будем хранить полученную информацию
for text_input in tqdm(df['text']):
    try:
        messages = prompt.format_messages(
            text=text_input,
            format_instructions=format_instructions
        )
        response = llm(messages) # Ответ модели
        dict_list.append(output_parser.parse(response.content)) # Переводим содержимое ответа модели в словарь и добавляем в список
    except:
        pass
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 50/50 [01:39<00:00,  1.98s/it]


In [ ]:
# dict_list

In [74]:
result_df = pd.DataFrame(dict_list) # Создаем датафрейм из полученной информации
ans_df = pd.concat([df, result_df], axis=1) # Объединяем всё в один датафрейм
ans_df.to_csv("../data/2_2_9_solution.csv", index=False)

In [75]:
ans_df

,text_id,text,job_title,company,salary,tg,grade
0,9,#вакансия #vacancy #Python #удаленка #flask #r...,Python developer,Collectly,6000-9000$,@ann_gfio,senior
1,31,#ВАКАНСИЯ #Системный_аналитик #РФ\n \nАКЦИОНЕ...,СИСТЕМНОГО АНАЛИТИКА,АКЦИОНЕРНОЕ ОБЩЕСТВО «ГЛАВНЫЙ НАУЧНЫЙ ИННОВАЦИ...,320000-370000 рублей,@NatalyaMaki,
2,28,#вакансия #vacancy #job #senior #data #DB #dat...,Database Administrator,Match Systems,от 3000$,@lex_kertis,Senior
3,49,#вакансия #fulltime #remote \n\n🔎 Ищем Руковод...,Руководитель отдела системного администрирования,финтех продукт,от 3000$,@ResearcherRIT,
4,18,#vacancy #job #analyst #travel #sirenatravel #...,Analyst,Sirena Travel,120000-180000 руб.,@ann_gfio,
5,48,#вакансия #fulltime #remote #lookfor #devops\n...,DevOps инженер,Mad Devs,до 5000$,@recruiter_maddevs,Senior
6,41,#вакансия #senior #middle+ #lead #удаленка #оф...,Системный аналитик,Платформа,180000-300000 руб.,@Alexandrabogdanova_96,middle+
7,6,#job #python #django #javascript #react #fulls...,Python/Django full-stack разработчика,ivelum,$4000 - $6000,@lebedevaoi,"middle, senior"
8,17,#вакансия #vacancy #middle #senior #remote #уд...,Go разработчик,КА Алешин Д.А.,до 500000 руб.,@AleshinDmitry80,"middle, senior"
9,4,Вакансия: Разработчик (Vue.js / Golang) для на...,Разработчик (Vue.js / Golang),,,,
